# Análisis de Datos para Tablero de Power BI

Este notebook analiza los datos de proyecciones de población e indicadores de pobreza para preparar la información necesaria para un tablero en Power BI.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurar visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

## 1. Cargar los Datos

Cargaremos los tres archivos de Excel disponibles para su análisis.

In [ ]:
# Definir rutas a los archivos
current_dir = os.getcwd()
file_pob = os.path.join(current_dir, 'anex-DCD-Proypoblacion-PerteneniaEtnicoRacialmun.xlsx')
file_pm = os.path.join(current_dir, 'anex-PM-Departamental-2023.xlsx')
file_pmd = os.path.join(current_dir, 'anex-PMultidimensional-Departamental-2024.xlsx')

print(f"Archivos a cargar:\n{file_pob}\n{file_pm}\n{file_pmd}")

In [ ]:
# Función auxiliar para mostrar las hojas de un archivo Excel
def show_excel_sheets(file_path):
    try:
        xl = pd.ExcelFile(file_path)
        print(f"Hojas en {os.path.basename(file_path)}:")
        for sheet in xl.sheet_names:
            print(f"  - {sheet}")
        return xl.sheet_names
    except Exception as e:
        print(f"Error al leer {file_path}: {e}")
        return None

In [ ]:
# Ver hojas disponibles en cada archivo
sheets_pob = show_excel_sheets(file_pob)
sheets_pm = show_excel_sheets(file_pm)
sheets_pmd = show_excel_sheets(file_pmd)

## 2. Cargar y Limpiar Datos de Proyecciones de Población

In [ ]:
def clean_data_population(df):
    # Renombrar columnas si es necesario
    if 'Unnamed: 0' in df.columns:
        # Asumiendo una estructura similar a la vista en el notebook original
        column_mapping = {
            'Unnamed: 0': 'COD_DPTO',
            'Unnamed: 1': 'DEPARTAMENTO',
            'Unnamed: 2': 'COD_DPTO-MPIO',
            'Unnamed: 3': 'MUNICIPIO',
            'Unnamed: 4': 'AÑO',
            'Unnamed: 5': 'ÁREA GEOGRÁFICA',
            'Unnamed: 6': 'Total',
            'Unnamed: 7': 'Indígena',
            'Unnamed: 8': 'Gitano(a) o Rrom',
            'Unnamed: 9': 'Raizal del Archipiélago de San Andrés, Providencia y Santa Catalina',
            'Unnamed: 10': 'Palenquero(a) de San Basilio',
            'Unnamed: 11': 'Negro(a), mulato(a), afrodescendiente, afrocolombiano(a)',
            'Unnamed: 12': 'Ningún grupo étnico-racial'
        }
        df = df.rename(columns=column_mapping)
    
    # Eliminar filas iniciales que suelen ser encabezados o notas
    # Ajustar según sea necesario después de revisar los datos reales
    try:
        df = df.dropna(subset=['COD_DPTO'], how='all')
    except:
        # Si no existe la columna, probablemente necesitemos un enfoque diferente
        pass
    
    return df

# Intentamos cargar los datos con diferentes parámetros de skiprows
try:
    # Primero intentamos cargar sin skiprows para ver la estructura
    df_pob_raw = pd.read_excel(file_pob)
    print("Primeras filas del archivo sin procesar:")
    display(df_pob_raw.head())
    
    # Ahora intentamos con skiprows basado en la observación de las primeras filas
    for skip_rows in [0, 5, 10]:
        print(f"\nIntentando con skiprows={skip_rows}")
        df_pob = pd.read_excel(file_pob, skiprows=skip_rows)
        display(df_pob.head(2))
        
    # Aplicamos la limpieza a la versión que parece más apropiada
    # Ajustar el número de filas a omitir según lo observado
    df_pob = pd.read_excel(file_pob, skiprows=10)  # Ajustar según observación
    df_pob_clean = clean_data_population(df_pob)
    print("\nDatos limpios:")
    display(df_pob_clean.head())
except Exception as e:
    print(f"Error al cargar datos de población: {e}")

## 3. Cargar Datos de Pobreza Monetaria

In [ ]:
# Cargar los indicadores clave de pobreza monetaria
def load_poverty_indicators(file_path, sheet_names):
    indicators = {}
    for sheet in sheet_names:
        if sheet != 'Índice':  # Omitir la hoja de índice
            try:
                # Cargar cada hoja y guardarla en un diccionario
                # Ajustar skiprows según sea necesario después de revisar los datos
                df = pd.read_excel(file_path, sheet_name=sheet, skiprows=5)
                # Limpiar el dataframe
                df = df.dropna(how='all').dropna(how='all', axis=1)
                indicators[sheet] = df
                print(f"Cargado {sheet} con forma {df.shape}")
            except Exception as e:
                print(f"Error al cargar {sheet}: {e}")
    return indicators

# Intentar cargar los datos de pobreza monetaria
try:
    pm_indicators = load_poverty_indicators(file_pm, sheets_pm)
    
    # Mostrar un ejemplo de uno de los indicadores cargados
    if pm_indicators and len(pm_indicators) > 0:
        example_key = list(pm_indicators.keys())[0]
        print(f"\nEjemplo de datos para {example_key}:")
        display(pm_indicators[example_key].head())
except Exception as e:
    print(f"Error general al cargar indicadores de pobreza monetaria: {e}")

## 4. Cargar Datos de Pobreza Multidimensional

In [ ]:
# Cargar los indicadores de pobreza multidimensional
try:
    pmd_indicators = load_poverty_indicators(file_pmd, sheets_pmd)
    
    # Mostrar un ejemplo de uno de los indicadores cargados
    if pmd_indicators and len(pmd_indicators) > 0:
        example_key = list(pmd_indicators.keys())[0]
        print(f"\nEjemplo de datos para {example_key}:")
        display(pmd_indicators[example_key].head())
except Exception as e:
    print(f"Error general al cargar indicadores de pobreza multidimensional: {e}")

## 5. Identificar Departamento Asignado

La actividad menciona que deben realizar un tablero para el "departamento asignado". Deberá identificar cuál es su departamento asignado.

In [ ]:
# Lista de departamentos disponibles
# Extraer de alguno de los dataframes cargados
try:
    if 'IPM_Departamentos' in pmd_indicators:
        # Intentar extraer la lista de departamentos de la hoja de departamentos
        dept_df = pmd_indicators['IPM_Departamentos']
        if 'Departamento' in dept_df.columns:
            departments = dept_df['Departamento'].dropna().unique().tolist()
        else:
            # Intentar encontrar la columna que contiene nombres de departamentos
            potential_dept_columns = [col for col in dept_df.columns if 'depart' in col.lower()]
            if potential_dept_columns:
                departments = dept_df[potential_dept_columns[0]].dropna().unique().tolist()
            else:
                # Si no encontramos una columna específica, mostrar todas las columnas
                print("Columnas disponibles:")
                print(dept_df.columns.tolist())
                departments = []
    else:
        # Intentar extraer de otro dataframe
        example_key = list(pmd_indicators.keys())[0]  # Tomar cualquier hoja disponible
        dept_df = pmd_indicators[example_key]
        # Buscar columnas que puedan contener nombres de departamentos
        potential_dept_columns = [col for col in dept_df.columns if any(term in col.lower() for term in ['depart', 'dpto', 'región'])]
        if potential_dept_columns:
            departments = dept_df[potential_dept_columns[0]].dropna().unique().tolist()
        else:
            # Si no encontramos una columna específica, mostrar todas las columnas
            print("Columnas disponibles:")
            print(dept_df.columns.tolist())
            departments = []
    
    print("\nDepartamentos disponibles:")
    for i, dept in enumerate(departments, 1):
        print(f"{i}. {dept}")
    
    # Aquí deberá seleccionar su departamento asignado
    # Por ahora, tomaremos el primero como ejemplo
    if departments:
        assigned_department = departments[0]
        print(f"\nPara este ejemplo, trabajaremos con el departamento: {assigned_department}")
except Exception as e:
    print(f"Error al identificar departamentos: {e}")

## 6. Generar Datos para el Tablero de Power BI

Ahora procesaremos los datos para cada uno de los indicadores solicitados en la actividad, filtrados para el departamento asignado.

In [ ]:
# Función para filtrar datos por departamento
def filter_by_department(df, dept_column, department):
    """Filtra un dataframe por un departamento específico"""
    if dept_column in df.columns:
        return df[df[dept_column] == department]
    else:
        print(f"Columna {dept_column} no encontrada. Columnas disponibles: {df.columns.tolist()}")
        return df

# Definir el departamento a analizar (debe reemplazarse con el departamento asignado)
try:
    # Si no se pudo identificar automáticamente, definir manualmente
    if 'assigned_department' not in locals() or not assigned_department:
        assigned_department = "VALLE DEL CAUCA"  # Departamento asignado para este análisis
    
    print(f"Preparando datos para el departamento: {assigned_department}")
    
    # Crear directorio para almacenar los datos procesados si no existe
    output_dir = os.path.join(current_dir, 'datos')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Directorio creado: {output_dir}")
except Exception as e:
    print(f"Error al preparar el entorno: {e}")

In [ ]:
# Procesar los datos de proyección de población
try:
    if 'df_pob_clean' in locals() and not df_pob_clean.empty:
        # Encontrar la columna de departamento
        dept_col = [col for col in df_pob_clean.columns if 'DEPART' in col.upper()]
        if dept_col:
            dept_col = dept_col[0]
            # Filtrar por departamento
            dept_pop = filter_by_department(df_pob_clean, dept_col, assigned_department)
            print(f"Datos de población para {assigned_department}: {dept_pop.shape[0]} registros")
            
            # Guardar para Power BI
            pop_output_file = os.path.join(output_dir, f'poblacion_{assigned_department}.csv')
            dept_pop.to_csv(pop_output_file, index=False)
            print(f"Datos guardados en: {pop_output_file}")
        else:
            print("No se pudo identificar la columna de departamento en los datos de población")
    else:
        print("No se pudieron cargar correctamente los datos de proyección de población")
except Exception as e:
    print(f"Error al procesar datos de población: {e}")

In [ ]:
# Procesar indicadores de pobreza monetaria
try:
    if 'pm_indicators' in locals() and pm_indicators:
        # Crear un dataframe consolidado para Power BI
        pm_consolidated = pd.DataFrame()
        
        for indicator_name, df in pm_indicators.items():
            # Buscar columna de departamento
            potential_dept_cols = [col for col in df.columns if any(term in str(col).lower() for term in ['depart', 'dpto', 'entidad'])]
            
            if potential_dept_cols:
                dept_col = potential_dept_cols[0]
                # Filtrar por departamento
                filtered_df = df[df[dept_col].str.contains(assigned_department, case=False, na=False)]
                
                if not filtered_df.empty:
                    # Renombrar columnas para claridad
                    filtered_df = filtered_df.rename(columns={col: f"{indicator_name}_{col}" 
                                                            for col in filtered_df.columns 
                                                            if col != dept_col})
                    
                    # Guardar individualmente
                    indicator_file = os.path.join(output_dir, f'{indicator_name}_{assigned_department}.csv')
                    filtered_df.to_csv(indicator_file, index=False)
                    print(f"Indicador {indicator_name} guardado en: {indicator_file}")
                    
                    # Si es el primer indicador, inicializar el dataframe consolidado
                    if pm_consolidated.empty:
                        pm_consolidated = filtered_df
                    else:
                        # Intentar unir con el dataframe consolidado
                        # Esto puede requerir ajustes según la estructura real de los datos
                        try:
                            pm_consolidated = pd.merge(pm_consolidated, filtered_df, on=dept_col, how='outer')
                        except Exception as merge_error:
                            print(f"No se pudo unir {indicator_name}: {merge_error}")
            else:
                print(f"No se pudo identificar columna de departamento en {indicator_name}")
        
        # Guardar el dataframe consolidado
        if not pm_consolidated.empty:
            consolidated_file = os.path.join(output_dir, f'pobreza_monetaria_consolidado_{assigned_department}.csv')
            pm_consolidated.to_csv(consolidated_file, index=False)
            print(f"Datos consolidados de pobreza monetaria guardados en: {consolidated_file}")
    else:
        print("No se pudieron cargar correctamente los indicadores de pobreza monetaria")
except Exception as e:
    print(f"Error al procesar indicadores de pobreza monetaria: {e}")

In [ ]:
# Procesar indicadores de pobreza multidimensional
try:
    if 'pmd_indicators' in locals() and pmd_indicators:
        # Crear un dataframe consolidado para Power BI
        pmd_consolidated = pd.DataFrame()
        
        for indicator_name, df in pmd_indicators.items():
            # Buscar columna de departamento
            potential_dept_cols = [col for col in df.columns if any(term in str(col).lower() for term in ['depart', 'dpto', 'entidad'])]
            
            if potential_dept_cols:
                dept_col = potential_dept_cols[0]
                # Filtrar por departamento
                filtered_df = df[df[dept_col].str.contains(assigned_department, case=False, na=False)]
                
                if not filtered_df.empty:
                    # Renombrar columnas para claridad
                    filtered_df = filtered_df.rename(columns={col: f"{indicator_name}_{col}" 
                                                            for col in filtered_df.columns 
                                                            if col != dept_col})
                    
                    # Guardar individualmente
                    indicator_file = os.path.join(output_dir, f'{indicator_name}_{assigned_department}.csv')
                    filtered_df.to_csv(indicator_file, index=False)
                    print(f"Indicador {indicator_name} guardado en: {indicator_file}")
                    
                    # Si es el primer indicador, inicializar el dataframe consolidado
                    if pmd_consolidated.empty:
                        pmd_consolidated = filtered_df
                    else:
                        # Intentar unir con el dataframe consolidado
                        try:
                            pmd_consolidated = pd.merge(pmd_consolidated, filtered_df, on=dept_col, how='outer')
                        except Exception as merge_error:
                            print(f"No se pudo unir {indicator_name}: {merge_error}")
            else:
                print(f"No se pudo identificar columna de departamento en {indicator_name}")
        
        # Guardar el dataframe consolidado
        if not pmd_consolidated.empty:
            consolidated_file = os.path.join(output_dir, f'pobreza_multidimensional_consolidado_{assigned_department}.csv')
            pmd_consolidated.to_csv(consolidated_file, index=False)
            print(f"Datos consolidados de pobreza multidimensional guardados en: {consolidated_file}")
    else:
        print("No se pudieron cargar correctamente los indicadores de pobreza multidimensional")
except Exception as e:
    print(f"Error al procesar indicadores de pobreza multidimensional: {e}")

## 7. Visualizaciones de Ejemplo para Power BI

Para ayudar en la creación del tablero, generamos algunas visualizaciones de ejemplo de los indicadores clave.

In [ ]:
# Crear visualizaciones de ejemplo
try:
    # Definir una paleta de colores para consistencia visual
    colors = sns.color_palette("viridis", 10)
    
    # Configurar el tamaño de las figuras
    plt.figure(figsize=(12, 8))
    
    # 1. Gráfico de ejemplo para Incidencia de Pobreza Monetaria y Extrema por año
    if 'pm_indicators' in locals() and 'IP Act.Met.' in pm_indicators and 'IPE Act.Met.' in pm_indicators:
        ip_df = pm_indicators['IP Act.Met.']
        ipe_df = pm_indicators['IPE Act.Met.']
        
        # Identificar columnas relevantes
        year_cols_ip = [col for col in ip_df.columns if str(col).isdigit()]
        dept_cols_ip = [col for col in ip_df.columns if any(term in str(col).lower() for term in ['depart', 'dpto', 'entidad'])]
        
        if dept_cols_ip and year_cols_ip:
            dept_col_ip = dept_cols_ip[0]
            # Filtrar por departamento
            ip_dept = ip_df[ip_df[dept_col_ip].str.contains(assigned_department, case=False, na=False)]
            
            if not ip_dept.empty:
                # Crear gráfico de tendencia
                plt.figure(figsize=(10, 6))
                plt.title(f'Incidencia de Pobreza Monetaria - {assigned_department}')
                
                # Datos para gráfico
                years = [int(col) for col in year_cols_ip]
                values = ip_dept[year_cols_ip].values[0]
                
                plt.plot(years, values, marker='o', linestyle='-', color=colors[0], linewidth=2)
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.xlabel('Año')
                plt.ylabel('Incidencia (%)')
                plt.xticks(years)
                
                # Guardar gráfico
                plt.savefig(os.path.join(output_dir, f'incidencia_pobreza_{assigned_department}.png'))
                plt.close()
                print(f"Gráfico de incidencia de pobreza guardado en el directorio de datos")
    
    # 2. Gráfico de Coeficiente de Gini
    if 'pm_indicators' in locals() and 'Gini' in pm_indicators:
        gini_df = pm_indicators['Gini']
        
        # Identificar columnas relevantes
        year_cols_gini = [col for col in gini_df.columns if str(col).isdigit()]
        dept_cols_gini = [col for col in gini_df.columns if any(term in str(col).lower() for term in ['depart', 'dpto', 'entidad'])]
        
        if dept_cols_gini and year_cols_gini:
            dept_col_gini = dept_cols_gini[0]
            # Filtrar por departamento
            gini_dept = gini_df[gini_df[dept_col_gini].str.contains(assigned_department, case=False, na=False)]
            
            if not gini_dept.empty:
                # Crear gráfico de barras
                plt.figure(figsize=(10, 6))
                plt.title(f'Coeficiente de Gini - {assigned_department}')
                
                # Datos para gráfico
                years = [int(col) for col in year_cols_gini]
                values = gini_dept[year_cols_gini].values[0]
                
                plt.bar(years, values, color=colors)
                plt.grid(True, linestyle='--', alpha=0.7, axis='y')
                plt.xlabel('Año')
                plt.ylabel('Coeficiente de Gini')
                plt.xticks(years)
                
                # Guardar gráfico
                plt.savefig(os.path.join(output_dir, f'gini_{assigned_department}.png'))
                plt.close()
                print(f"Gráfico de Gini guardado en el directorio de datos")
    
    # 3. Gráfico para Pobreza Multidimensional
    if 'pmd_indicators' in locals() and 'IPM_Departamentos' in pmd_indicators:
        ipm_df = pmd_indicators['IPM_Departamentos']
        
        # Identificar columnas relevantes
        year_cols_ipm = [col for col in ipm_df.columns if str(col).isdigit()]
        dept_cols_ipm = [col for col in ipm_df.columns if any(term in str(col).lower() for term in ['depart', 'dpto', 'entidad'])]
        
        if dept_cols_ipm and year_cols_ipm:
            dept_col_ipm = dept_cols_ipm[0]
            # Filtrar por departamento
            ipm_dept = ipm_df[ipm_df[dept_col_ipm].str.contains(assigned_department, case=False, na=False)]
            
            if not ipm_dept.empty:
                # Crear gráfico de línea
                plt.figure(figsize=(10, 6))
                plt.title(f'Incidencia de Pobreza Multidimensional - {assigned_department}')
                
                # Datos para gráfico
                years = [int(col) for col in year_cols_ipm]
                values = ipm_dept[year_cols_ipm].values[0]
                
                plt.plot(years, values, marker='s', linestyle='-', color=colors[2], linewidth=2)
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.xlabel('Año')
                plt.ylabel('Incidencia (%)')
                plt.xticks(years)
                
                # Guardar gráfico
                plt.savefig(os.path.join(output_dir, f'incidencia_pobreza_multidimensional_{assigned_department}.png'))
                plt.close()
                print(f"Gráfico de pobreza multidimensional guardado en el directorio de datos")
except Exception as e:
    print(f"Error al generar visualizaciones: {e}")

## 8. Recomendaciones para Power BI

Con los datos procesados en el directorio 'datos', ahora puede importarlos a Power BI para crear su tablero. Aquí hay algunas recomendaciones:

1. **Importación de datos**: Importe los archivos CSV generados en el directorio 'datos'

2. **Estructura sugerida para su tablero**:
   - Página 1: Resumen de indicadores clave para el departamento asignado
   - Página 2: Análisis de tendencias de pobreza monetaria 
   - Página 3: Análisis de pobreza multidimensional
   - Página 4: Análisis por características demográficas (género, área, etc.)
   - Página 5: Proyecciones de población

3. **Visualizaciones recomendadas**:
   - Tarjetas para mostrar los valores más recientes de indicadores clave
   - Gráficos de líneas para mostrar tendencias a lo largo del tiempo
   - Gráficos de barras para comparar diferentes indicadores
   - Mapas para visualizar datos geográficos si están disponibles
   - Tablas para mostrar datos detallados

4. **Interactividad**:
   - Use filtros para permitir la selección por año, área geográfica, etc.
   - Implemente segmentaciones de datos para filtrar múltiples visualizaciones a la vez
   - Configure interacciones entre visualizaciones para análisis profundos

5. **Estética y diseño**:
   - Use una paleta de colores consistente
   - Incluya títulos descriptivos en todas las visualizaciones
   - Agregue texto explicativo para contextualizar los datos
   - Mantenga un diseño limpio y organizado